<a href="https://colab.research.google.com/github/prateekrana17/SPARK-NLP-RAG-LLM/blob/main/SPARK-NLP-latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh


--2024-05-15 06:17:02--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘colab_setup.sh’

colab_setup.sh      100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-05-15 06:17:02 (37.9 MB/s) - ‘colab_setup.sh’ saved [1191/1191]



In [3]:
!bash colab_setup.sh

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.7 MB/s eta 0:00:00


In [5]:
spark = sparknlp.start()
spark

In [6]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [7]:
# detection model_name = 'ld_wiki_cnn_231'
model_name = 'ld_wiki_tatoeba_cnn_375'

In [60]:
text = """Die Mona Lisa ist ein von Leonardo geschaffenes Ölgemälde aus dem 16. Jahrhundert. Sie findet im Louvre in Paris statt."""  #document/ ECM retireved text

text2 = """Ich liebe Datenwissenschaf"""

In [61]:
# create spark data frame

data = spark.createDataFrame([
    [text],
    [text2]]).toDF("text")

In [62]:
# detection pipeline for 100+ languages

# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)


documentAssembler = DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")

sentence_detector = SentenceDetector() \
            .setInputCols(["document"]) \
            .setOutputCol("sentence")

# Step 2: Determines the language of the text
languageDetector = LanguageDetectorDL.pretrained(model_name)\
            .setInputCols("sentence")\
            .setOutputCol("language")\
            .setThreshold(0.5)\
            .setCoalesceSentences(True)

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentence_detector,
        languageDetector])


ld_wiki_tatoeba_cnn_375 download started this may take some time.
Approximate size to download 8.8 MB
[OK!]


In [64]:
#language detection
result = nlpPipeline.fit(data).transform(data)
result.select("text", "language.result").show()

+--------------------+------+
|                text|result|
+--------------------+------+
|Die Mona Lisa ist...|  [de]|
|Ich liebe Datenwi...|  [de]|
+--------------------+------+



In [49]:
# Tranlsation
t_list= []

In [65]:
# Translation pipeline
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

## More accurate Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")

marian = MarianTransformer.pretrained("opus_mt_it_en", "xx")\
  .setInputCols(["sentences"])\
  .setOutputCol("translation")

nlp_pipeline = Pipeline(
    stages=[
        documentAssembler,
        sentencerDL,
        marian
        ])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_it_en download started this may take some time.
Approximate size to download 454.8 MB
[OK!]


IllegalArgumentException: requirement failed: Wrong or missing inputCols annotators in LANGUAGE_DETECTOR_DL_b307513d73d0.

Current inputCols: sentence. Dataset's columns:
(column_name=text,is_nlp_annotator=false)
(column_name=document,is_nlp_annotator=true,type=document).
Make sure such annotators exist in your pipeline, with the right output names and that they have following annotator types: document

In [42]:
#create spark Data Frame for displaying detection results
df = spark.createDataFrame(user_text, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

In [25]:
# Tranlation pipeline


## More accurate Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")


marian = MarianTransformer.pretrained("opus_mt_it_en", "xx")\
  .setInputCols(["sentences"])\
  .setOutputCol("translation")

nlp_pipeline2 = Pipeline(
    stages=[
        documentAssembler,
        sentencerDL,
        marian
        ])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_it_en download started this may take some time.
Approximate size to download 454.8 MB
[OK!]


In [26]:
# translation dataframe

empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = nlp_pipeline2.fit(empty_df)
lmodel = LightPipeline(pipeline_model)
res = lmodel.fullAnnotate(translation_text)


[{'document': [Annotation(document, 0, 4338, Mona Lisa is a 16th century oil painting created by Leonardo. It is held at the Louvre in Paris. La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris. Джефри Еверест Хинтън е британски канадски когнитивен психолог и компютърен учен, най-известен с работата си върху изкуствени невронни мрежи. От 2013 г. той прекарва времето си в работа за Google и университета в Торонто. През 2017 г. е съосновател и става главен научен съветник на Vector Institute of Toronto. Mona Lisa je olejomalba ze 16. století, kterou vytvořil Leonardo. Koná se v Louvru v Paříži. Titanic ist ein 1997 in den USA veröffentlichter epischer Roman und ein katastrophaler Film, der von James Cameron inszeniert, geschrieben, co-produziert und mitherausgegeben wurde. Es deckt sowohl historische als auch fiktive Aspekte ab und basiert auf Berichten über den Untergang der RMS Titanic und der Stars Leonard DiCaprio und Kate Winslet als